In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import os
from torchvision.datasets import ImageFolder

In [2]:
# Define transformations
transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
root_dir = 'C:\\Users\\gguid\\Desktop\\Colpo\\ColpoData'

In [4]:
# Define the valid class names
valid_classes = ['0 Normal', '1 Ectropion', '2 Metaplasia', '3 Bajo grado', '4 Alto grado']

# Filter out non-image files and folders and exclude specified folders
filtered_classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d)) and d in valid_classes and not d.startswith('.')]

# Define dataset and apply transformations
classified_dataset = ImageFolder(root=root_dir, transform=transform, is_valid_file=lambda f: f.endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp')))


In [5]:
# Split already classified data into training and validation sets
train_data, val_data = train_test_split(classified_dataset, test_size=0.2, random_state=42)


In [6]:
# Define data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

# Define the model
#model = torchvision.models.resnet18(pretrained=True)
#num_ftrs = model.fc.in_features
#model.fc = nn.Linear(num_ftrs, 5)  # 3 output classes

# Define the model with updated parameters
model = torchvision.models.resnet18(weights="ResNet18_Weights.DEFAULT")
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Assuming you have 5 output classes

In [7]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'image_classifier.pth')

Epoch 1/20, Loss: 1.5415
Epoch 2/20, Loss: 1.1352
Epoch 3/20, Loss: 0.6944
Epoch 4/20, Loss: 0.5536
Epoch 5/20, Loss: 0.5389
Epoch 6/20, Loss: 0.4412
Epoch 7/20, Loss: 0.4904
Epoch 8/20, Loss: 0.3695
Epoch 9/20, Loss: 0.2676
Epoch 10/20, Loss: 0.1887
Epoch 11/20, Loss: 0.1621
Epoch 12/20, Loss: 0.2512
Epoch 13/20, Loss: 0.1544
Epoch 14/20, Loss: 0.0726
Epoch 15/20, Loss: 0.0574
Epoch 16/20, Loss: 0.1098
Epoch 17/20, Loss: 0.1620
Epoch 18/20, Loss: 0.1672
Epoch 19/20, Loss: 0.1414
Epoch 20/20, Loss: 0.0847


In [9]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

# Define transformations for the test images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the trained model
model = torchvision.models.resnet18(pretrained=False)  # Load the same architecture as trained
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Assuming 5 output classes
model.load_state_dict(torch.load('image_classifier.pth'))  # Load the trained weights
model.eval()  # Set the model to evaluation mode

# Define the directory containing the test images
quiz_dir = 'C:\\Users\\gguid\\Desktop\\Colpo\\quiz'

# Iterate through each image in the directory
for filename in os.listdir(quiz_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        # Load and preprocess the image
        image_path = os.path.join(quiz_dir, filename)
        image = Image.open(image_path)
        image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Classify the image
        with torch.no_grad():
            output = model(image_tensor)
            _, predicted = torch.max(output, 1)

        # Get the class label
        class_label = predicted.item()

        # Map class index to class name
        class_mapping = {0: '0 Normal', 1: '1 Ectropion', 2: '2 Metaplasia', 3: '3 Bajo grado', 4: '4 Alto grado'}
        predicted_class = class_mapping[class_label]

        # Print the name of the image and its classification
        print(f"Image: {filename}, Classification: {predicted_class}")


C:\Users\gguid\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gguid\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Image: 1.1.p.0.jpg, Classification: 0 Normal
Image: 2.1.p.0.jpg, Classification: 4 Alto grado
Image: 2.2.p.0.jpg, Classification: 4 Alto grado
Image: 3.1.n.0.jpg, Classification: 4 Alto grado
Image: 3.1.p.h2-3.jpg, Classification: 4 Alto grado
Image: 3.2.p.h2-3.jpg, Classification: 4 Alto grado
Image: 4.1.p.h2-3.jpg, Classification: 4 Alto grado
Image: 4.2.p.h2-3.jpg, Classification: 4 Alto grado
Image: 5.1.p.h2-3.jpg, Classification: 4 Alto grado
Image: 5.2.p.h2-3.jpg, Classification: 4 Alto grado
Image: 5.3.p.h2-3.jpg, Classification: 4 Alto grado
Image: 6.1.p.ca.jpg, Classification: 4 Alto grado
Image: 7.1.n.metap.jpg, Classification: 0 Normal
Image: 7.2.n.metap.jpg, Classification: 4 Alto grado
Image: 8.1.n.0.jpg, Classification: 3 Bajo grado
Image: 9.1.p.h2-3.jpg, Classification: 4 Alto grado
Image: 99.1.n.0.jpg, Classification: 4 Alto grado
Image: 99.2.n.0.jpg, Classification: 4 Alto grado
Image: 991.1.p.ca.jpg, Classification: 4 Alto grado
Image: 992.1.n.metap.jpg, Classificatio

In [12]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the trained model
model = torchvision.models.resnet18(pretrained=False)  # Load the same architecture as trained
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 output classes
model.load_state_dict(torch.load('image_classifier.pth'))  # Load the trained weights
model.eval()  # Set the model to evaluation mode

# Define transformations for the new image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the new image
image_path = 'C:\\Users\\gguid\\Desktop\\Colpo\\ColpoData\\0 Normal\\4.jpg'
image = Image.open(image_path)
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Classify the image
with torch.no_grad():
    output = model(image_tensor)
    _, predicted = torch.max(output, 1)

# Get the class label
class_label = predicted.item()

# Print the predicted class
class_mapping = {0: 'Normal', 1: 'Ectropion', 2: 'Metaplasia'}
predicted_class = class_mapping[class_label]
print("Predicted class:", predicted_class)


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([5, 512]) from checkpoint, the shape in current model is torch.Size([3, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([3]).